In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [22]:
#Import weather data
weather_data_df = pd.read_csv("weather_cities.csv",usecols=["City","Lat","Lng","Max Temp",\
                                              "Humidity","Cloudiness","Wind Speed","Country"]) 
weather_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,aswan,24.09,32.91,100.24,13,6,17.56,EG
1,lishui,28.46,119.91,78.78,85,88,1.90,CN
2,pisco,-13.70,-76.22,64.00,82,40,8.05,PE
3,namatanai,-3.67,152.43,78.24,83,100,4.32,PG
4,punta arenas,-53.15,-70.92,41.00,55,100,20.80,CL
...,...,...,...,...,...,...,...,...
570,sabang,5.89,95.32,84.33,73,23,15.32,ID
571,lumberton,34.62,-79.01,91.00,55,20,10.29,US
572,lavrentiya,65.58,-171.00,45.82,83,24,5.37,RU
573,solhan,38.96,41.03,72.59,24,3,2.80,TR


In [49]:
#Configure gmaps
gmaps.configure(api_key=g_key)
locations = weather_data_df[["Lat","Lng"]]
weight = weather_data_df["Humidity"].astype(float)
fig = gmaps.figure(zoom_level=2,center=(25,0))


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Add heat layer
fig.add_layer(heat_layer)

# Display
fig

Figure(layout=FigureLayout(height='420px'))

In [47]:
#new criteria dataframe

weather_criteria_df = weather_data_df[weather_data_df.Humidity <= 85]
weather_criteria_df= weather_criteria_df[weather_criteria_df["Humidity"] >= 45]
weather_criteria_df= weather_criteria_df[weather_criteria_df["Wind Speed"] <= 10]
weather_criteria_df= weather_criteria_df[weather_criteria_df["Max Temp"] <= 80]
weather_criteria_df= weather_criteria_df[weather_criteria_df["Max Temp"] >= 70]
weather_criteria_df= weather_criteria_df[weather_criteria_df["Cloudiness"] <= 35]
weather_criteria_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
102,chuy,-33.70,-53.46,71.38,66,26,4.14,UY
106,lagoa,39.05,-27.98,70.90,66,2,3.33,PT
109,hanstholm,57.12,8.62,70.00,69,2,5.06,DK
140,orange,33.70,-117.77,78.01,64,1,4.70,US
193,sulina,45.16,29.65,75.42,55,0,6.82,RO


In [48]:
#Hotel
weather_criteria_df["Hotel Name"] = ""
weather_criteria_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
102,chuy,-33.70,-53.46,71.38,66,26,4.14,UY,
106,lagoa,39.05,-27.98,70.90,66,2,3.33,PT,
109,hanstholm,57.12,8.62,70.00,69,2,5.06,DK,
140,orange,33.70,-117.77,78.01,64,1,4.70,US,
193,sulina,45.16,29.65,75.42,55,0,6.82,RO,
210,nikolskoye,59.70,30.79,71.60,60,20,6.71,RU,
231,fuling,29.70,107.39,78.08,82,0,3.13,CN,
266,torbay,47.67,-52.73,71.60,68,20,9.17,CA,
276,rabo de peixe,37.80,-25.58,75.20,69,20,2.24,PT,
291,la palma,33.85,-118.05,78.01,56,1,6.40,US,


In [72]:
#Hotel
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
parameters = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
for index, row in weather_criteria_df.iterrows():
    coordinates = f'{row["Lat"]},{row["Lng"]}'
    parameters["location"] = coordinates
    response = requests.get(base_url, params=parameters).json()
    weather_criteria_df.loc[index, 'Hotel Name'] = results[0]['name']

IndexError: list index out of range

In [65]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in weather_criteria_df.iterrows()]
coordinates = weather_criteria_df[["Lat", "Lng"]]

In [64]:
markers = gmaps.marker_layer(coordinates,info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

ValueError: not enough values to unpack (expected 2, got 1)